In [2]:
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms, models, datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torch import optim
device = 'cpu'
import cv2, glob, pandas as pd
%matplotlib inline
from glob import glob

In [4]:
train_data_dir = 'archive/training_set/training_set'
test_data_dir = 'archive/test_set/test_set'

In [6]:
from torch.utils.data import Dataset, DataLoader
class cats_dogs(Dataset):
    def __init__(self,folder):
        cats =  glob(folder+'/cats/*.jpg')
        dogs =  glob(folder+'/dogs/*.jpg')
        self.fpaths = cats + dogs
        from random import shuffle, seed; seed(10); shuffle(self.fpaths)
        self.targets = [fpath.split('/')[-1].startswith('dog') for fpath in self.fpaths] # dog=1, cat=0
    def __len__(self): 
        return len(self.fpaths)
    def __getitem__(self, ix):
        f = self.fpaths[ix]
        target =  self.targets[ix]
        im = (cv2.imread(f)[:,:,::-1])
        im = cv2.resize(im, (224,224))
        return torch.tensor(im/255).permute(2,0,1).to(device).float(), torch.tensor([target]).float().to(device)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary

device = 'cpu'

# Função para criar uma camada convolucional com mais regularização
def conv_layer(ni, no, kernel_size, stride=1):
    return nn.Sequential(
        nn.Conv2d(ni, no, kernel_size, stride),
        nn.ReLU(),
        nn.BatchNorm2d(no),
        nn.MaxPool2d(2),
        nn.Dropout(0.3)  # Adicionando Dropout para regularização
    )

# Função para obter o modelo
def get_model():
    model = nn.Sequential(
        conv_layer(3, 32, 3),   # Reduzi o número de filtros na primeira camada
        conv_layer(32, 64, 3),  # Mantive a progressão dos filtros
        conv_layer(64, 128, 3), # Aumentei a complexidade nas camadas intermediárias
        conv_layer(128, 256, 3),
        conv_layer(256, 512, 3),
        nn.Flatten(),
        nn.Linear(512*7*7, 256),  # Ajustando a dimensão da entrada para a camada densa
        nn.ReLU(),
        nn.Dropout(0.5),           # Dropout adicional na camada densa
        nn.Linear(256, 1),
        nn.Sigmoid()
    ).to(device)
    
    loss_fn = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Reduzi o learning rate
    return model, loss_fn, optimizer

# Função para treinar um batch
def train_batch(x, y, model, optimizer, loss_fn):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()  # Zerando os gradientes após a atualização
    return batch_loss.item()

# Resumo do modelo
model, loss_fn, optimizer = get_model()
summary(model, input_size=(3, 224, 224))


Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       896
|    └─ReLU: 2-2                         --
|    └─BatchNorm2d: 2-3                  64
|    └─MaxPool2d: 2-4                    --
|    └─Dropout: 2-5                      --
├─Sequential: 1-2                        --
|    └─Conv2d: 2-6                       18,496
|    └─ReLU: 2-7                         --
|    └─BatchNorm2d: 2-8                  128
|    └─MaxPool2d: 2-9                    --
|    └─Dropout: 2-10                     --
├─Sequential: 1-3                        --
|    └─Conv2d: 2-11                      73,856
|    └─ReLU: 2-12                        --
|    └─BatchNorm2d: 2-13                 256
|    └─MaxPool2d: 2-14                   --
|    └─Dropout: 2-15                     --
├─Sequential: 1-4                        --
|    └─Conv2d: 2-16                      295,168
|    └─ReLU: 2-17                        --
|    └─Batc

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       896
|    └─ReLU: 2-2                         --
|    └─BatchNorm2d: 2-3                  64
|    └─MaxPool2d: 2-4                    --
|    └─Dropout: 2-5                      --
├─Sequential: 1-2                        --
|    └─Conv2d: 2-6                       18,496
|    └─ReLU: 2-7                         --
|    └─BatchNorm2d: 2-8                  128
|    └─MaxPool2d: 2-9                    --
|    └─Dropout: 2-10                     --
├─Sequential: 1-3                        --
|    └─Conv2d: 2-11                      73,856
|    └─ReLU: 2-12                        --
|    └─BatchNorm2d: 2-13                 256
|    └─MaxPool2d: 2-14                   --
|    └─Dropout: 2-15                     --
├─Sequential: 1-4                        --
|    └─Conv2d: 2-16                      295,168
|    └─ReLU: 2-17                        --
|    └─Batc